In [1]:
import psycopg2
import psycopg2.extras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import figure
import dataframe_image as dfi

In [2]:
connection = psycopg2.connect(
    database="root",
    user="root",
    password="password",
    host="127.0.0.1",
    port="5440"
)
db = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)


In [3]:
def getAll(query):
    db.execute(query)
    return pd.DataFrame(data=db.fetchall())

In [4]:
# top 10 userów którzy napisali najwięcej tweetów

sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba tweetów"
from "user" u
         join tweet t on u.id = t.user_id
where t.reply_to is null
  and t.type = 1
group by u.id, u.name
order by count(*) desc
limit 10;
"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-tweetow.png")
df1


,Nazwa użytkownika,Liczba tweetów
0,PulpNews Crime,341
1,informazione interno,321
2,The Hill,262
3,FR24 NEWS,260
4,Medical Zirigoza Group,255
5,Today's news summary,238
6,Morning News 24/7,220
7,REFORMA,188
8,Noticieros Televisa,186
9,La Patilla,185


In [5]:
# top 10 userów którzy napisali najwięcej komentarzy

sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba komentarzy"
from "user" u
         join tweet t on u.id = t.user_id
where t.type = 2
group by u.id, u.name
order by count(*) desc
limit 10;
"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-komentarzy.png")
df1

,Nazwa użytkownika,Liczba komentarzy
0,GetVideoBot,79
1,GetVideoBot 🤖,70
2,DownloadThisVideo🤖,61
3,Neena Jha 💙,46
4,Megan Ranney MD MPH 🗽,40
5,Tatiana Schild,40
6,Nicholas A. Christakis,38
7,Old Spice LA,37
8,Video Downloader Bot,32
9,Terry,30


In [6]:
# top 10 userów którzy napisali najwięcej pozytywnych tweetow


sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba pozytywnych tweetow"
from "user" u
         join tweet t on u.id = t.user_id
where t.reply_to is null
  and t.type = 1
  and t.sentiment_compound >= 0.05
group by u.id, u.name
order by count(*) desc
limit 10;
"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-pozytywnych-tweetow.png")
df1

,Nazwa użytkownika,Liczba pozytywnych tweetow
0,P&O Ferries Freight,160
1,Henry Niman,137
2,david pietrzak,104
3,GlobalPandemic.NET,93
4,PulpNews Crime,91
5,The Hill,91
6,SpeechTrans,87
7,Today's news summary,69
8,Morning News 24/7,64
9,Con Sweeney,59


In [7]:
# top 10 userów którzy napisali najwięcej negatywnych tweetow


sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba negatywnych tweetow"
from "user" u
         join tweet t on u.id = t.user_id
where t.reply_to is null
  and t.type = 1
  and t.sentiment_compound <= -0.05
group by u.id, u.name
order by count(*) desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-negatywnych-tweetow.png")
df1

,Nazwa użytkownika,Liczba negatywnych tweetow
0,Covid Counting,82
1,Eric Feigl-Ding,80
2,The Hill,75
3,Morning News 24/7,71
4,Today's news summary,68
5,PulpNews Crime,62
6,Reuters,59
7,CNN,56
8,Ray Giles,55
9,COVIDBot,55


In [8]:
# top 10 userów którzy napisali najwięcej pozytywnych komentarzy


sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba pozytywnych komentarzy"
from "user" u
         join tweet t on u.id = t.user_id
where t.type = 2
  and t.sentiment_compound >= 0.05
group by u.id, u.name
order by count(*) desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-pozytywnych-komentarzy.png")
df1

,Nazwa użytkownika,Liczba pozytywnych komentarzy
0,GetVideoBot,67
1,GetVideoBot 🤖,42
2,Neena Jha 💙,27
3,Terry,24
4,Thread Reader App,24
5,DownloadThisVideo🤖,23
6,Nicholas A. Christakis,20
7,Def Noodles,18
8,Glasgow Women's Library,18
9,Pizza Inn,16


In [9]:
# top 10 userów którzy napisali najwięcej negatywnych komentarzy

sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba negatywnych komentarzy"
from "user" u
         join tweet t on u.id = t.user_id
where t.type = 2
  and t.sentiment_compound <= -0.05
group by u.id, u.name
order by count(*) desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-negatywnych-komentarzy.png")
df1

,Nazwa użytkownika,Liczba negatywnych komentarzy
0,Jeremy Konyndyk,15
1,i. 🧈,14
2,Old Spice LA,14
3,Megan Ranney MD MPH 🗽,14
4,Frink,12
5,Nicholas A. Christakis,11
6,Tatiana Schild,11
7,Andy Slavitt 🇺🇸💉,10
8,COVID19 Physicians,9
9,#STAYHEALTHY&SAFE #BlackLivesMatter,8


In [10]:
# top 10 najpopularniejszych tweetow z największą ilością komentarzy + retweetów + cytowań + polubień

sql1 = """
select t.id as "Identyfikator tweeta", t.content, t.reply_count + t.retweet_count + t.quote_count + t.favorite_count as "Suma komentarzy, retweetów cytowań i polubień"
from tweet t
where t.type = 1
order by "Suma komentarzy, retweetów cytowań i polubień" desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-najpopularniejszych-tweetow.png")
df1

,Identyfikator tweeta,content,"Suma komentarzy, retweetów cytowań i polubień"
0,1351897267666608129,It’s a new day in America.,4860898
1,1266821069823975430,Congratulations to the Astronauts that left Ea...,4457232
2,1265832328598884352,What a headline https://t.co/x8GGKL0VMc,3628671
3,1266156916121706502,"It’s not white vs black, it’s everybody vs rac...",2043345
4,1236029449042198528,The coronavirus panic is dumb,1976320
5,1331307848647761925,America is back.,1899480
6,1325890910635384839,"I won't be president until January 20th, but m...",1710610
7,1342491135093207044,I’m going to give 5 people that retweet this t...,1700489
8,1239834212565430278,We. Are. The. Problem.,1692453
9,1239617034901524481,This morning I tested positive for Covid 19. I...,1682337
